In [1]:
import csv
import numpy as np
from random import shuffle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import sklearn
import cv2

samples = []

# Steering angle correction in order to add left and right pictures. 
# Maximum angle is 25 degrees so we want a float multiplying that number
# It seems the difference between side and center pictures is about 20 degrees
# that would imply a float multiplier of 0.8 for the side pictures if we wanted immediate correction
# However 0.8 seems too much so we'll start with lower than that
corrections = ['0.0', '0.2', '-0.2']


with open('data/cdriving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        center_info = [line[0], line[3], corrections[0]]
        left_info = [line[1], line[3], corrections[1]]
        right_info = [line[2], line[3], corrections[2]]
        samples.append(center_info)
        samples.append(left_info)
        samples.append(right_info)
    
print("samples is of length:",len(samples))

       
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'data/IMG/'+batch_sample[0].split('/')[-1]
                image = mpimg.imread(name)
                # Cropping and resizing the image using cv2
                mod_image = image[68:134, :]
                imagecv = cv2.resize(mod_image,(200,66), interpolation = cv2.INTER_AREA)
                angle = float(batch_sample[1]) + float(batch_sample[2])
                images.append(imagecv)
                angles.append(angle)
                
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
                
                
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32) 

samples is of length: 41790


In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(66, 200,3)))
model.add(Convolution2D(24,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(36,5,5, subsample=(2, 2), activation="relu"))
model.add(Convolution2D(48,5,5, subsample=(2, 2), activation="relu"))
model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
    
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), 
                      validation_data=validation_generator, 
                      nb_val_samples=len(validation_samples), nb_epoch=4)

# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model.save('modelnew.h5')

Epoch 1/4
33432/33432 [==============================] - 466s - loss: 0.0184 - val_loss: 0.0161

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 66, 200, 3)    0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 62, 196, 24)   1824        lambda_2[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 31, 98, 24)    0           convolution2d_6[0][0]            
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 14, 47, 36)    21636       maxpooling2d_2[0][0]             
___________________________________________________________________________________________